In [14]:
import torch
import librosa
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Загрузка модели
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

def split_audio(audio_path, min_silence_len=500, silence_thresh=-40, keep_silence=200):
    """Разбиваем аудио на фрагменты по тишине"""
    audio = AudioSegment.from_wav(audio_path)
    chunks = split_on_silence(
        audio,
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh,
        keep_silence=keep_silence
    )
    return chunks

def process_chunk(chunk, chunk_idx):
    """Обработка одного аудиофрагмента"""
    # Экспортируем фрагмент во временный файл
    chunk.export(f"temp_chunk_{chunk_idx}.wav", format="wav")
    
    # Загружаем как массив numpy
    speech_array, sr = librosa.load(f"temp_chunk_{chunk_idx}.wav", sr=16_000)
    
    # Обработка через модель
    inputs = processor(speech_array, sampling_rate=sr, return_tensors="pt", padding=True)
    
    with torch.no_grad():
        logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
    
    predicted_ids = torch.argmax(logits, dim=-1)
    return processor.batch_decode(predicted_ids)[0]

def analyze_audio(audio_path):
    """Основная функция анализа"""
    print(f"\nAnalyzing: {audio_path}")
    
    # Разбиваем на фрагменты
    chunks = split_audio(audio_path)
    print(f"Found {len(chunks)} chunks")
    
    # Обрабатываем каждый фрагмент
    full_text = []
    for i, chunk in enumerate(chunks):
        try:
            text = process_chunk(chunk, i)
            print(f"Chunk {i+1}: {text}")
            full_text.append(text)
        except Exception as e:
            print(f"Error processing chunk {i+1}: {str(e)}")
    
    # Объединяем результаты
    return " ".join(full_text)

if __name__ == "__main__":
    audio_paths = ["IMG_5264.wav"]  # Ваши аудиофайлы
    
    for audio_path in audio_paths:
        try:
            result = analyze_audio(audio_path)
            print("\nFinal Result:")
            print(result)
        except Exception as e:
            print(f"Error analyzing {audio_path}: {str(e)}")


Analyzing: IMG_5264.wav
Found 12 chunks
Chunk 1: 
Chunk 2: я вас любил
Chunk 3: любовь еще быть может
Chunk 4: в душе моей угаслы не совсем
Chunk 5: но пусть она вас больше не тревожит
Chunk 6: я не хочу печалить вас ничем
Chunk 7: я вас любил безмогвно безнадежно
Chunk 8: то робостью то ревностью томим
Chunk 9: я вас любил так искренне так нежно
Chunk 10: 
Chunk 11: когдай вам бог любимый быть другим
Chunk 12: 

Final Result:
 я вас любил любовь еще быть может в душе моей угаслы не совсем но пусть она вас больше не тревожит я не хочу печалить вас ничем я вас любил безмогвно безнадежно то робостью то ревностью томим я вас любил так искренне так нежно  когдай вам бог любимый быть другим 


In [ ]:

LANG_ID = "ru"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
SAMPLES = 5

test_dataset = load_dataset("common_voice", LANG_ID, split=f"test[:{SAMPLES}]")

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["path"], sr=16_000)
    batch["speech"] = speech_array
    batch["sentence"] = batch["sentence"].upper()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)

for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Reference:", test_dataset[i]["sentence"])
    print("Prediction:", predicted_sentence)
